In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/final_proj

/content/drive/.shortcut-targets-by-id/1YMOPkl5pAMR5Y440QTaZiAbd1YXrDEUb/final_proj


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import os
import pickle
import numpy as np
import numpy.random as rng
from tensorflow.keras import layers, models
from keras import backend as K
from keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model

In [ ]:
train_in = open("mini-imagenet-cache-train.pkl", "rb")
train = pickle.load(train_in)
ungrouped_Xtrain = train["image_data"]
val_in = open("mini-imagenet-cache-val.pkl", "rb")
val = pickle.load(val_in)
ungrouped_Xval = val["image_data"]

In [ ]:
train_mean = ungrouped_Xtrain.mean(axis=(0,1,2)) 
train_std = ungrouped_Xtrain.std(axis=(0,1,2))

ungrouped_Xtrain = ungrouped_Xtrain.astype('float32')
ungrouped_Xval = ungrouped_Xval.astype('float32')

In [ ]:
ungrouped_Xtrain -= train_mean
ungrouped_Xtrain /= train_std
ungrouped_Xval -= train_mean
ungrouped_Xval /= train_std

In [ ]:
train_data = ungrouped_Xtrain.reshape([64, 600, 84, 84, 3])
val_data = ungrouped_Xval.reshape([16, 600, 84, 84, 3])
train_data.shape

(64, 600, 84, 84, 3)

In [ ]:
def get_pairwise_batch(batch_size, train_data, input_shape):
    """
    Create batch of n pairs, half same class, half different class
    """
    n_classes, n_examples, w, h, d = train_data.shape
    new_w, new_h, new_d = input_shape
    

    rng = np.random.default_rng()

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,))
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, new_w, new_h, new_d)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)
        pairs[0][i,:,:,:] = tf.image.resize(train_data[category, idx_1].reshape(w, h, d), (224, 224)).numpy()
        idx_2 = np.random.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = tf.image.resize(train_data[category_2,idx_2].reshape(w, h, d), (224, 224)).numpy()
    
    return pairs, targets


def data_generator(data, batch_size, input_shape):
    while True:
        (inputs,targets) = get_pairwise_batch(batch_size, train_data, input_shape)
        yield inputs, targets

In [ ]:
def fn_loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y, preds):
      # explicitly cast the true class label data type to the predicted
      # class label data type (otherwise we run the risk of having two
      # separate data types, causing TensorFlow to error out)
      y = tf.cast(y, preds.dtype)
      # calculate the contrastive loss between the true labels and
      # the predicted labels
      squaredPreds = K.square(preds)
      squaredMargin = K.square(K.maximum(margin - preds, 0))
      loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
      # return the computed contrastive loss to the calling function
      return loss

    return contrastive_loss

In [ ]:
def L2_Norm(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
      keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
class BottleneckResidualBlock(layers.Layer):
  def __init__(self, expansion_factor, channels, stride, name=None, trainable=True):
    super(BottleneckResidualBlock, self).__init__(name=name, trainable=trainable)
    self.expansion_factor = expansion_factor
    self.channels = channels
    self.stride = stride

  def build(self, input_shape):
    input_channels = input_shape[3]
    self.c1 = layers.Conv2D(filters=self.expansion_factor*input_channels, kernel_size=(1,1), use_bias=False)
    self.dc1 = layers.DepthwiseConv2D(kernel_size=(3,3), strides=self.stride, use_bias=False, padding='same')
    self.c2 = layers.Conv2D(filters=self.channels, kernel_size=(1,1), use_bias=False)

    self.bn1 = layers.BatchNormalization()
    self.bn2 = layers.BatchNormalization()
    self.bn3 = layers.BatchNormalization()

  def call(self, inputs, training=False):
    x = self.c1(inputs)
    x = tf.nn.relu6(x)
    x = self.bn1(x, training=training)

    x = self.dc1(x)
    x = tf.nn.relu6(x)
    x = self.bn2(x, training=training)
    
    x = self.c2(x)
    x = self.bn3(x, training=training)
    
    if self.stride == 1 and x.shape[1:] == inputs.shape[1:]:
      x += inputs

    return x

  def get_config(self):
    config = super().get_config()
    config.update({
        "expansion_factor": self.expansion_factor,
        "channels": self.channels,
        "stride": self.stride,
    })
    return config

In [ ]:
input_shape = (224, 224, 3)
input_l = layers.Input(input_shape)
input_r = layers.Input(input_shape)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), strides=2, padding='same', activation=tf.nn.relu6, input_shape=input_shape))
model.add(layers.BatchNormalization())

# bottleneck 1
model.add(BottleneckResidualBlock(1, 16, 1))
# bottleneck 2
model.add(BottleneckResidualBlock(6, 24, 2))
model.add(BottleneckResidualBlock(6, 24, 1))
# bottleneck 3
model.add(BottleneckResidualBlock(6, 32, 2))
for i in range(2):
  model.add(BottleneckResidualBlock(6, 32, 1))
# bottleneck 4
model.add(BottleneckResidualBlock(6, 64, 2))
for i in range(3):
  model.add(BottleneckResidualBlock(6, 64, 1))
# bottleneck 5
for i in range(3):
  model.add(BottleneckResidualBlock(6, 96, 1))
# bottleneck 6
model.add(BottleneckResidualBlock(6, 160, 2))
for i in range(2):
  model.add(BottleneckResidualBlock(6, 160, 1))
# bottleneck 7
model.add(BottleneckResidualBlock(6, 320, 1))

model.add(layers.Conv2D(1280, (1, 1), activation=tf.nn.relu6))
model.add(layers.BatchNormalization())

model.add(layers.AveragePooling2D((7, 7)))

model.add(layers.Conv2D(1280, (1, 1), activation=tf.nn.relu6))
model.add(layers.BatchNormalization())

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.1))

encoded_l = model(input_l)
encoded_r = model(input_r)

L2_distance = layers.Lambda(L2_Norm)([encoded_l, encoded_r])
model = Model(inputs=[input_l,input_r],outputs=L2_distance)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 1280)         3905248     ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

In [ ]:
# Hyper params
lr = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.045,
    decay_steps=10000,
    decay_rate=0.98)
momentum = 0.9
margin = 1

In [ ]:
# Hyper parameters
evaluate_every = 1000  # interval for evaluating on one-shot tasks\
# download_every = 1000
batch_size = 64
n_iter = 10000 # No. of training iterations

# used for one batching testing
N_way = 10 # how many classes for testing one-shot tasks. has to be less than num classes in dataset
n_val = 250 # how many one-shot tasks to validate on

# used for straight validation testing
val_batch_size = 64

val_test_size = 8000

best = -1

steps_per_epoch = 1000
epochs = 10

In [ ]:
optimizer = RMSprop(learning_rate = lr, momentum=momentum)
model.compile(loss=fn_loss(margin=margin),optimizer=optimizer)

In [ ]:
history = model.fit(
    data_generator(train_data, batch_size, (224, 224, 3)),
    steps_per_epoch=steps_per_epoch,
    validation_data = data_generator(val_data, val_batch_size, (224, 224, 3)),
    validation_steps = 10,
    epochs=epochs, verbose=True)

Epoch 1/10
1000/1000 [==============================] - 2127s 2s/step - loss: 5.4544 - val_loss: 35223.2031
Epoch 2/10
1000/1000 [==============================] - 2095s 2s/step - loss: 0.5461 - val_loss: 32063.0664
Epoch 3/10
1000/1000 [==============================] - 2099s 2s/step - loss: 0.3492 - val_loss: 186053.9062
Epoch 4/10
1000/1000 [==============================] - 2098s 2s/step - loss: 0.3474 - val_loss: 113805.3359
Epoch 5/10
1000/1000 [==============================] - 2098s 2s/step - loss: 0.3477 - val_loss: 428352.3438
Epoch 6/10
1000/1000 [==============================] - 2090s 2s/step - loss: 0.3727 - val_loss: 85758.4219
Epoch 7/10
1000/1000 [==============================] - 2078s 2s/step - loss: 0.3483 - val_loss: 1140857.7500
Epoch 8/10
1000/1000 [==============================] - 2087s 2s/step - loss: 0.3459 - val_loss: 454497.0625
Epoch 9/10
1000/1000 [==============================] - 2091s 2s/step - loss: 0.3467 - val_loss: 257163.5312
Epoch 10/10
1000/1000

In [ ]:
model_path = './weights/'
model_name = "mobilenet_contrastive_2"

In [ ]:
model.save_weights(os.path.join(model_path, 'weights_{}.h5'.format(model_name)))